In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

dtype = torch.FloatTensor

In [8]:
char_arr = [c for c in 'abcdefghijklmnopqrstuvwxyz'] # ['a', 'b', 'c',...]
word2idx = {n: i for i, n in enumerate(char_arr)}
idx2word = {i: w for i, w in enumerate(char_arr)}
n_class = len(word2idx) # number of class(=number of vocab)

seq_data = ['make', 'need', 'coal', 'word', 'love', 'hate', 'live', 'home', 'hash', 'star']

# TextLSTM Parameters
n_step = len(seq_data[0]) - 1 # (=3)
n_hidden = 128

In [9]:
def make_data(seq_data):
    input_batch, target_batch = [], []

    for seq in seq_data:
        input = [word2idx[n] for n in seq[:-1]] # 'm', 'a' , 'k' is input
        target = word2idx[seq[-1]] # 'e' is target
        input_batch.append(np.eye(n_class)[input])
        target_batch.append(target)

    return torch.Tensor(input_batch), torch.LongTensor(target_batch)

input_batch, target_batch = make_data(seq_data)
dataset = Data.TensorDataset(input_batch, target_batch)
loader = Data.DataLoader(dataset, 3, True)

c:\users\neko_maru~\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
  # Remove the CWD from sys.path while we load stuff.


In [10]:
class TextLSTM(nn.Module):
    def __init__(self):
        super(TextLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=n_class, hidden_size=n_hidden)
        self.fc = nn.Linear(n_hidden, n_class)
        
    def forward(self, X):
        # X: [batch_size, n_step, n_class]
        batch_size = X.shape[0]
        input = X.transpose(0, 1)  # X : [n_step, batch_size, n_class]
        
        hidden_state = torch.zeros(1, batch_size, n_hidden)   # [num_layers(=1) * num_directions(=1), batch_size, n_hidden]
        cell_state = torch.zeros(1, batch_size, n_hidden)    # [num_layers(=1) * num_directions(=1), batch_size, n_hidden]
        
        outputs, (_, _) = self.lstm(input, (hidden_state, cell_state)) # [max_len, batch_size, n_hidden]
        outputs = outputs[-1]  # [batch_size, n_hidden]
        model = self.fc(outputs)  # model : [batch_size, n_class]
        return model
    
model = TextLSTM()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [11]:
# Training
for epoch in range(1000):
    for x, y in loader:
        pred = model(x)
        loss = criterion(pred, y)
    if (epoch + 1) % 100 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

Epoch: 0100 cost = 0.618966
Epoch: 0200 cost = 0.847887
Epoch: 0300 cost = 0.180129
Epoch: 0400 cost = 0.011737
Epoch: 0500 cost = 0.109197
Epoch: 0600 cost = 0.007679
Epoch: 0700 cost = 0.326748
Epoch: 0800 cost = 0.035026
Epoch: 0900 cost = 0.000677
Epoch: 1000 cost = 0.004325


In [12]:
inputs = [sen[:3] for sen in seq_data]
predict = model(input_batch).data.max(1, keepdim=True)[1]
print(inputs, '->', [idx2word[n.item()] for n in predict.squeeze()])

['mak', 'nee', 'coa', 'wor', 'lov', 'hat', 'liv', 'hom', 'has', 'sta'] -> ['e', 'd', 'l', 'd', 'e', 'e', 'e', 'e', 'h', 'r']
